<a href="https://colab.research.google.com/github/hadiidfajri/Big-Data/blob/main/Regression_Product_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas numpy scikit-learn category_encoders matplotlib seaborn joblib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 4.2 MB/s eta 0:00:00


In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nadyinky/sephora-products-and-skincare-reviews")

print("Path to dataset files:", path)

100%|██████████| 147M/147M [00:01<00:00, 109MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/nadyinky/sephora-products-and-skincare-reviews/versions/2


In [4]:
import os

# List files in the downloaded directory
file_list = os.listdir(path)
print("Files in the downloaded directory:", file_list)

# Assuming the CSV file has a similar name to the dataset or a common name
# You might need to adjust the filename based on the actual contents of the directory
csv_filename = "reviews_1250-end.csv"

if csv_filename:
    csv_filepath = os.path.join(path, csv_filename)
    df = pd.read_csv(csv_filepath)
    display(df.head()) # Display the head of the dataframe to confirm it's loaded
else:
    print("No CSV file found in the downloaded directory.")
    df = None # Set df to None if no CSV is found

Files in the downloaded directory: ['reviews_0-250.csv', 'reviews_750-1250.csv', 'reviews_1250-end.csv', 'reviews_250-500.csv', 'reviews_500-750.csv', 'product_info.csv']


/tmp/ipython-input-2537766933.py:13: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_filepath)


,Unnamed: 0,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
0,0,1945004256,5,1.0,0.000000,2,2,0,2022-12-10,I absolutely L-O-V-E this oil. I have acne pro...,A must have!,lightMedium,green,combination,NaN,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0
1,1,5478482359,3,1.0,0.333333,3,2,1,2021-12-17,I gave this 3 stars because it give me tiny li...,it keeps oily skin under control,mediumTan,brown,oily,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0
2,2,29002209922,5,1.0,1.000000,2,0,2,2021-06-07,Works well as soon as I wash my face and pat d...,Worth the money!,lightMedium,brown,dry,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0
3,3,7391078463,5,1.0,1.000000,2,0,2,2021-05-21,"this oil helped with hydration and breakouts, ...",best face oil,lightMedium,brown,combination,blonde,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0
4,4,1766313888,5,1.0,1.000000,13,0,13,2021-03-29,This is my first product review ever so that s...,Maskne miracle,mediumTan,brown,combination,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0


In [5]:
df.shape

(49977, 19)

In [6]:
modus_isrecommended=df["is_recommended"].mode()[0]
df['is_recommended']=df['is_recommended'].fillna(modus_isrecommended)

In [7]:
df['helpfulness']=df['helpfulness'].fillna(0)

In [8]:
columns_to_impute_mode = ["skin_type","hair_color","eye_color","skin_tone"]

for column in columns_to_impute_mode :
  mode_value=df[column].mode()[0]
  df[column]=df[column].fillna(mode_value)
  print(f"Missing values in {column} : {df[column].isnull().sum()}")



Missing values in skin_type : 0
Missing values in hair_color : 0
Missing values in eye_color : 0
Missing values in skin_tone : 0


In [9]:


X = df[['brand_name', 'product_name', 'rating', 'helpfulness']]
y = df['price_usd']


In [10]:


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)


In [18]:
preprocessor = ColumnTransformer([
    ('brand_te', TargetEncoder(), ['brand_name']),
    ('product_tfidf', TfidfVectorizer(max_features=50), 'product_name'),
    ('scaler', RobustScaler(), ['rating', 'helpfulness'])
])

In [19]:

lasso_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', Lasso(alpha=0.1, random_state=42))
])

lasso_pipeline.fit(X_train, y_train)
lasso_pred = lasso_pipeline.predict(X_test)

lasso_r2 = r2_score(y_test, lasso_pred)
lasso_rmse = np.sqrt(mean_squared_error(y_test, lasso_pred))


In [20]:


rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(
        n_estimators=200,
        random_state=42
    ))
])

rf_pipeline.fit(X_train, y_train)
rf_pred = rf_pipeline.predict(X_test)

rf_r2 = r2_score(y_test, rf_pred)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))


In [21]:

lasso_mse = mean_squared_error(y_test, lasso_pred)
rf_mse = mean_squared_error(y_test, rf_pred)

print("📊 PERBANDINGAN HASIL MODEL")
print("====================================")
print(f"LASSO REGRESSION -> R²: {lasso_r2:.3f} | MSE: {lasso_mse:.3f} | RMSE: {lasso_rmse:.3f}")
print(f"RANDOM FOREST    -> R²: {rf_r2:.3f} | MSE: {rf_mse:.3f} | RMSE: {rf_rmse:.3f}")

📊 PERBANDINGAN HASIL MODEL
LASSO REGRESSION -> R²: 0.672 | MSE: 1794.988 | RMSE: 42.367
RANDOM FOREST    -> R²: 0.967 | MSE: 179.059 | RMSE: 13.381


In [22]:

new_product = pd.DataFrame({
    'brand_name': ['Fenty Beauty'],
    'product_name': ['Lip Tint Gloss'],
    'rating': [4.6],
    'helpfulness': [0.78]
})

lasso_price = lasso_pipeline.predict(new_product)[0]
rf_price = rf_pipeline.predict(new_product)[0]

print("\n PREDIKSI HARGA PRODUK BARU")
print("------------------------------------")
print(f"Produk : {new_product['product_name'][0]}")
print(f"Brand  : {new_product['brand_name'][0]}")
print(f"Lasso Regression → Prediksi Harga: ${lasso_price:.2f}")
print(f"Random Forest    → Prediksi Harga: ${rf_price:.2f}")


 PREDIKSI HARGA PRODUK BARU
------------------------------------
Produk : Lip Tint Gloss
Brand  : Fenty Beauty
Lasso Regression → Prediksi Harga: $61.46
Random Forest    → Prediksi Harga: $26.00
